# Overview 
In this workshop, we will show you some common patterns for model evaluation. We will be evaluating a LLMs ability to do Q&A by:
1. Creating a validation test set
1. Define metrics that make sense for our usecase both qualitative and quantitative
1. Create a quality assurance rubric that an large language model (LLM) can use to grade qualitatively
1. Run our test Suite


## Building the Evaluation Framework
To build a strong testing framework, we need to start with a set of (mostly) human curated question / answer pairs. These will be our gold standard of what the model should be doing; the quality of these questions will drive the quality of our entire workload. Ideally, we need to manually create at least 100, if not 100’s of questions, carefully crafting correct answers to each. I strongly recommend that every generative AI project starts here because every minute spent building these test questions will pay back your investment 10 fold in time saved debugging and in quality of your final output. It’s also a great way to align your team on what the project is trying to accomplish.

Resist the temptation to grab a premade list! Using your own questions from your use case will make a huge difference.


When evaluating a prompt, we want **at least** 100 examples to run benchmarks on. One of the best ways to differentiate between a gen AI science project and a viable product is to count the number of automated tests. A handful of manual tests? Science Project. An automated system of hundreds of tests that runs every time you propose a change? Production ready.


## Pre-Requisites

Pre-requisites
This notebook requires permissions to:

create and delete Amazon IAM roles
create, update and delete Amazon S3 buckets
access Amazon Bedrock
access to Amazon OpenSearch Serverless
If running on SageMaker Studio, you should add the following managed policies to your role:

1. AmazonS3FullAccess
1. AmazonBedrockFullAccess
1. Custom policy for Amazon OpenSearch Serverless such as:
``` json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:*",
            "Resource": "*"
        }
    ]
}
```



# Setup
Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

In [ ]:
# Install all the required dependencies

%pip install -U opensearch-py==2.3.1
%pip install -U boto3==1.34.82
%pip install -U langchain==0.1.13
%pip install -U pandas==2.2.2

In [ ]:
# Restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Create Eval Dataset

This part is a little tricky and time consuming. **For the purpose of this workshop, we went ahead and created a dataset**. We did this by using the prompt below to Bedrock (Claude3 Sonnet) and replacing the {pga_handbook} variable with chunks of the handbook. We did this a total of 5 times to create 100 examples. We also did some data massaging to convert the tsv to csv so it could be easily used. **Then we went through each example and manually validated the Q&A answers.**

We recommend currating additional QnA examples manually to ensure a robust dataset. We also recommend you add to this eval dataset over time.

## Prompt Used
``` xml
You are PromptWizard, an AI assistant designed to build synthetic evaluation Q&A samples based on the content of a PGA rule book. Your knowledge comes solely from from the content the Official PGA Tour Rulebook shown below

<PGARuleBook>
{pga_handbook}
</PGARuleBook>

<objective>
Your objective is to generate 20 question and answer pairs using the pga rulebook that can be used to create an evaluation framework. 
</objective>



<output_format>
Question1\tAnswer1
Question2\tAnswer2
...
Question20\tAnswer20
</output_format>

<constraints>
- All questions and answers must be directly relevant to the context of the and official rules for PGA Tour events
- Make sure to write the question in a way that a user would ask the question. For example, "What are you supposed to do when the ball lands in a spectators cup?"
- Answers should be factual, clear, and concise based on the playbook information
- Ensure good variability in topics across the 20 Q&A pairs
- No made up information - answers must come from the playbook context
-  Separate the question and answer with a \t character. We are generating a TSV.
</constraints>
```

Next lets import our dataset into pandas and take a look at it!

In [ ]:
import pandas as pd

eval_df = pd.read_csv('pga_tour_qna_eval_dataset.csv')

eval_df

# Define Metrics for Benchmarking

In this step, we'll be defining the metrics we care about in order to benchmark both different prompts and models. 

There are two main types of evaluation metrics. Qualitative and quantitative. Find descriptions of each type below. 

### Quantitative 
Quantitative metrics involve numerical measurements that can objectively compare different models. These typically include accuracy, perplexity, speed, and resource efficiency, among others. They provide a clear, standardized way to measure certain aspects of an LLM's performance, such as how well it predicts the next word in a sequence or how quickly it generates responses. 

For quantitative benchmarks we will create the following
1. Latency

### Qualitative
On the other hand, qualitative metrics assess the more subjective aspects of LLM performance, including the coherence, relevancy, creativity of generated text, and adherence to ethical guidelines. These are often evaluated through human judgment via methods such as expert reviews or user studies, offering insights into the user experience and the contextual appropriateness of the model's outputs. While quantitative metrics can offer precise, measurable benchmarks, qualitative metrics are crucial for understanding the nuances and real-world effectiveness of LLMs. 

For qualitative evals we will want to consider
1. Coherence of response
2. Relevance of context passed to the model through RAG
3. Accuracy of response
4. Adherance to brand guidelines


## How do we gather qualitative metrics?

To gather qualitative metrics, we have two options. (1) Create a QA rubrik and give it to human evaluators or (2) Use that same rubrik and give it to an LLM to evaluate the responses. 

As a test suite gets larger, human evaluation becomes a bottleneck. Grading 500+ answers every time you make a change to a prompt is not scalable. Because of this, we'll opt to use an LLM to evaluate our responses. For poorly scoring responses, we can then manually check to see what's going on and fix the responses

## Lets create a grading prompt
Below you'll find a prompt that takes in the question, model response, correct answer, and a rubric that you will create to evaluate models output.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages.base import BaseMessage

# We start by defining a "grader prompt" template.
def build_grader_prompt(question, response, correct_answer, rubric) -> BaseMessage:
    prompt = f"""You will be provided an answer that an assistant gave to a question, and a rubric that instructs you on what makes the answer correct or incorrect.
    
    Here is the question asked.
    <question>{question}</question>
    
    Here is the response that the assistant gave to the question.
    <assistant_response>{response}</assistant_response>
    
    Here is the correct answer.
    <correct_answer>{correct_answer}</correct_answer>
    
    Here is the rubric on how to grade the assistant response.
    <rubric>{rubric}</rubric>
    
    An answer is correct if it entirely meets the rubric criteria, and is otherwise incorrect.
    First, think through whether the answer is correct or incorrect based on the rubric inside <thinking></thinking> tags. Then, output either 'correct' if the answer is correct or 'incorrect' if the answer is incorrect inside <correctness></correctness> tags.
    
    After thinking and providing a correctness score, use what's in the thinking tag to generate json that marks each of the following categories as True or False.
    
    categories
    - accurateAnswer
    - missingContext
    - helpful
    
    Place the json in <categorized_eval></categorized_eval> tags.
    """

    # First we will generate a prompt template using Langchain and the prompt above
    chat_template: ChatPromptTemplate = ChatPromptTemplate.from_messages([
        ("human", prompt)
    ])
        
    # Next, we will insert all the variables into into the prompt. 
    return chat_template.format_messages(
        question=question,
        response=response,
        correct_answer=correct_answer,
        rubric=rubric
    )
        
    


# !!! TODO

As a next step, your task is to come up with a QA grading rubrik. This rubric should accurately describe what you're looking for.

In [ ]:
# Fill in your rubric here
RUBRIC = '''
TODO: Fill this out
'''

# Call Q&A Bot To Get Responses
In the following section, we will reuse the chat bot created from the previous workshop to get responses from it. We will then use your rubric to grade the models performance.


To speed up validation, we will call our Q&A bot by submitting questions from our eval dataset into a thread pool

In [ ]:
SONNET_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
HAIKU_ID = "anthropic.claude-3-haiku-20240307-v1:0"

# To flip between different models, you can change these global variable.
MODEL_TO_USE = HAIKU_ID

REGION = 'us-west-2'

# TODO: Set this knowledge base value from the previous workshop 
KB_ID = '<FROM_PREVIOUS_WORKSHOP>'


bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=REGION)

In [ ]:
# Helper Functions

from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import boto3
import time

from langchain_community.chat_models import BedrockChat


def call_bedrock(request):
    client = BedrockChat(
        model_id=MODEL_TO_USE, 
        model_kwargs= {"temperature": 0.5, "top_k": 500}
    )
    
    response = client.invoke(request)
    return response

def ask_bedrock_llm_with_knowledge_base(query: str) -> str:
    start = time.time()
    
    model_arn: str = f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_TO_USE}'
    
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={ 'text': query },
        retrieveAndGenerateConfiguration= {
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': KB_ID,
                'modelArn': model_arn
            }
        },
    )

    generated_text = response['output']['text']
    
    latency = time.time() - start
    
    return {
        'modelResponse': generated_text,
        'latency': latency
    }

# This is a bit funky. We're dumping all the requests into a thread pool
# And storing the index for the order in which they were submitted. 
# Lastly, we're inserting them into the response array at their index to ensure order.
def call_threaded(requests, function):
    # Dictionary to map futures to their position
    future_to_position = {}
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Submit all requests and remember their order
        for i, request in enumerate(requests):
            future = executor.submit(function, request)
            future_to_position[future] = i
        
        # Initialize an empty list to hold the responses
        responses = [None] * len(requests)
        
        # As each future completes, assign its result to the correct position
        for future in as_completed(future_to_position):
            position = future_to_position[future]
            try:
                response = future.result()
                responses[position] = response
            except Exception as exc:
                print(f"Request at position {position} generated an exception: {exc}")
                responses[position] = None  # Or handle the exception as appropriate
        
    return responses

In [ ]:
from langchain_core.messages.ai import AIMessage

# Convert DataFrame to a list of dictionaries. This makes it easier to work with a thread pool.
input_records: list[dict] = eval_df.to_dict('records')

# Create prompts for all of our records.
requests: list[str] = [r['question'] for r in input_records]

# Call Bedrock threaded to speed up getting all our responses.
responses: list[dict] = call_threaded(requests, ask_bedrock_llm_with_knowledge_base)
    


# Run Evaluations

In [ ]:
# Construct grader prompt
grader_prompts = []

for i, r in enumerate(input_records):    
    question = r['question']
    response = responses[i]['modelResponse']
    correct_answer = r['answer']
    prompt: BaseMessage = build_grader_prompt(question, response, correct_answer, RUBRIC)
    grader_prompts.append(prompt)

    
# Lets change the model we're using to Sonnet. It's generally good practice to use 
# a larger and more sophisticated model for grading.
MODEL_TO_USE = SONNET_ID

# Call Bedrock threaded to speed up getting all our responses.
grades: list[AIMessage] = call_threaded(grader_prompts, call_bedrock)


In [ ]:
import re
import json

# Strip out the correctness grade
def extract_correctness(response):
    # Regular expression to extract everything inside of the sumologquery tags
    regex = r'<correctness>(.*?)</correctness>'
    # Perform the regex search
    matches = re.search(regex, response, re.DOTALL)
    # Extract the matched content, if any
    return matches.group(1).strip() if matches else None

# Strip out the reasoning
def extract_reasoning(response):
    # Regular expression to extract everything inside of the sumologquery tags
    regex = r'<thinking>(.*?)</thinking>'
    # Perform the regex search
    matches = re.search(regex, response, re.DOTALL)
    # Extract the matched content, if any
    return matches.group(1).strip() if matches else None
    

def format_grading_results(grade: str):
    reasoning: str = extract_reasoning(grade)
    correctness: str =  extract_correctness(grade)
    
    formatted_grading = {
        'reasoning': reasoning,
        'correctness': correctness
    }
        
    return formatted_grading


formatted_grades = [format_grading_results(g.content) for g in grades]


In [ ]:
evaluated_records = []

for i, r in enumerate(input_records):
    question = r['question']
    response = responses[i]['modelResponse']
    latency = responses[i]['latency']
    correct_answer = r['answer']
    correctness = formatted_grades[i]['correctness']
    reasoning = formatted_grades[i]['reasoning']
    
    eval_record = {
        'question': question,
        'correct_answer': correct_answer,
        'model_response': response,
        'correctness': correctness,
        'reasoning': reasoning,
        'latency': latency
    }
    
    evaluated_records.append(eval_record)
    
    
evaluated_df = pd.DataFrame(evaluated_records)
    

In [ ]:
evaluated_df

# Results

Now that we have our new evaluation dataframe, lets do an analysis on the results

In [ ]:
# First lets check out the latency

# Calculating statistics
average_latency = evaluated_df['latency'].mean()
median_latency = evaluated_df['latency'].median()
min_latency = evaluated_df['latency'].min()
max_latency = evaluated_df['latency'].max()

print("Average Latency:", average_latency)
print("Median Latency:", median_latency)
print("Lowest Latency:", min_latency)
print("Top Latency:", max_latency)

In [ ]:
# Next lets see how many we got correct

percentage_correct = evaluated_df['correctness'].value_counts(normalize=True)['correct'] * 100

print(f"Percentage correct: {percentage_correct:.2f}%")

In [ ]:
# Lastly we need to do some human evaluation. Lets sample a subsection of 10 incorrect responses
from IPython.display import display, HTML

# Assuming you have a dataframe called 'df' with a column called 'result'
incorrect_rows = evaluated_df[evaluated_df['correctness'] == 'incorrect'].sample(n=10)

from IPython.display import display, HTML

# Convert the dataframe to an HTML table
table_html = incorrect_rows.to_html(index=False, classes='table table-striped')

# Display the HTML table
display(HTML(table_html))

# Next Steps

Now that you've run through this notebook. 
* Go back and play with the rubric. 
* You can also play with the temperature and other hyperparameters of the model to see how that has an effect on your score.
* We used Haiku for the Q&A response. Set MODEL_TO_USE to SONNET_ID and rerun the test suite to see if that has an affect on the score.

In the next section of the workshop, we'll discuss advanced RAG techniques and how that could have an affect on performance. 